In [4]:
import pandas as pd

df = pd.read_csv("homepage_data.csv")
df2 = pd.read_csv("homepage_data_3.csv")
df = df[~df['Personal Website'].str.endswith('.sg').astype(bool)]
df = pd.concat([df, df2], ignore_index=True)
df.head()

,Name,Personal Website,About Us Text
0,Albert Chi-Shing CHUNG,https://cse.hkust.edu.hk/admin/people/faculty/...,Albert Chung is a Full Professor at the Depart...
1,Andrew Wing-On POON,https://eeawpoon.people.ust.hk/,Professor Poon received his BA degree from the...
2,Antoni Bert CHAN,https://www.cs.cityu.edu.hk/~abchan/,Dr. Antoni Chan is a Professor at the City Uni...
3,Brian Kan-Wing MAK,https://cse.hkust.edu.hk/admin/people/faculty/...,Dr. Mak is currently an Associate Professor in...
4,CHAN Chee Yong,https://sgps.cuhk.edu.hk/profile/professor-cha...,Professor Chee Hon Chan is an Assistant Profes...


In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
from markdownify import markdownify as md
import json

load_dotenv(".env")

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

def process_homepage_text(text: str):
    response = client.chat.completions.create(
        messages=[{
            "role": "system",
#             "content": """Extract the following qualities, with justification in a JSON formatted output. 
# Try to infer these from the way the author writes about themselves. Use and describe subtle language patterns to extract this understanding about a given individual. Try to look for small things like wordings, language choices, quotes, emphasis put on different things and related ideas. JSON output format: 
# {
#     "friendliness-justification": "Explain your friendliness quote with citations",
#     "friendliness": "Scale of 1-10, how much they exhibit traits of cameraderie, teamwork and collaboration. 10 being heavy-emphasis on friendliness.",
#     "we-vs-i-justification": "Explain your we-vs-i quote with citations",
#     "we-vs-i": "Scale of 1-10, how much they talk about themselves vs their collaborators. 10 being heavy-emphasis on we.",
#     "risk-averse-justification": "Explain your risk-averse quote with citations",
#     "risk-averse": "Scale of 1-10, how they feel about taking risks. 10 being very risk-averse.",
#     "career-orientation-justification": "Explain your career-orientation quote with citations",
#     "career-orientation": "How much a person cares about their career. 10 being very career-oriented.",
# }"""
        "content": """	•	SOCIAL CAMOUFLAGE

Definition
Social camouflage occurs when individuals, faced with the potential risks tied to particular identity markers (e.g., regional, ethnic, or other social affiliations) in a contentious situation, adjust their public and professional profiles to downplay or obscure that identity. This might involve distancing themselves from in-group markers, using neutral language about their origins, or highlighting connections to “safer” or more broadly accepted communities.

Theoretical Underpinnings
Individuals derive part of their self-concept from membership in social groups (Tajfel and Turner, 1979).  When an external event suddenly makes that group membership riskier (e.g., stigmatized, politicized, or closely monitored), people may seek to minimize exposure to adverse outcomes by attenuating the visible markers of that identity. Rather than celebrating or emphasizing one’s membership, individuals consciously or subconsciously adjust their behavior or affiliations to safeguard their professional or personal well-being. In this sense, camouflage manifests in observable behavior: professionals seek to protect themselves by changing who they collaborate with or how they present their identity cues.

People routinely manage the impressions they convey to others (Goffman, 1959; Elsbach and Kramer, 1996). In “camouflage” mode, researchers might adopt collaborative practices that mask, blur, or neutralize identity cues—similar to “passing” or “covering” (Goffman, 1963), which refers to the concealment or softening of a stigmatized attribute. These shifts need not be calculated or explicitly displayed publically; subtle reconfigurations of who is asked to join a project or which research topics are emphasized can serve a protective, identity-shielding function.

Notable aspects of social camouflage include:

	•	Behavioral adaptation. Social camouflage does not imply that every shift in collaboration patterns is deliberate and conscious. Instead, it captures the notion that when a particular identity feels risky due to heightened public attention, interpersonal tension, or institutional uncertainty, individuals may gravitate, sometimes even intuitively, toward pathways that reduce the salience of that identity in professional settings. 

	•	Subtle identity management. People can “camouflage” their identities in many ways, and collaboration patterns are just one of them. Even if no one scrutinizes a researcher’s co-author lists line by line, a slight shift in who is invited or included in a project may reflect a broader, often even subconscious, effort to minimize perceived vulnerabilities.

	•	Context-dependency. For some researchers, their academic environment, lab culture, or project funding sources may influence collaboration choices, whether or not they explicitly aim to influence outsiders’ perceptions. Social camouflage captures the possibility that social or regional identity risks subtly shape who teams up with whom—even if no one is sure others are paying close attention.

Current Empirical Operationalization
Social camouflage can be linked to retreating from or masking certain affiliations (e.g., reducing in-group partnerships or increasing out-group partnerships to blur one’s identity). For instance, if a certain regional identity becomes stigmatized, individuals might reduce their collaborations with others sharing that identity or increase projects with out-group members. In doing so, they effectively dilute the visibility of their “risky” group membership.

Some Ideas for Website Cues
Following Hong Kong’s Umbrella Movement in 2014—an event that heightened political and social sensitivities around local identity—some computer science professors may have felt pressure to alter how they present themselves. Social camouflage in this setting involves minimizing or obscuring markers of a Hong Kong affiliation to avoid potential scrutiny or stigmatization. Whether consciously or not, these academics might shift their self-descriptions to more neutral or global terms and reduce explicit references to a “Hong Kong” identity.

	•	De-emphasized in-group labels: Reduced mention of regional affiliations or explicit group memberships (e.g., changing “Hong Kong researcher” to “Asian researcher”).

	•	Shift in collaboration spotlight: Showcasing collaborations that cross typical group boundaries or leave out once-prominent ties to an in-group.

	•	Selective language about identity: Subtle disclaimers or ambiguous references to nationality, background, or social causes that could be associated with a charged identity.

For example, we could measure (1) reduced frequency or removal of in-group labels (e.g., dropping phrases like “I’m a Hong Kong researcher”) or (2) neutral, global, or mainland-Chinese-based reframing of one’s background (e.g., replacing “I am from X region” with “I’m based in Asia/the world” and/or “I’m based in China”). 

We could also measure indirect clues such as (3) replacing or downplaying once-prominent local society memberships (e.g., removing mention of a regional academic association) or (4) highlighting global/international/mainland Chinese associations.

*The differences among these three might be theoretically significant depending on what we empirically see. We could also combine them as a “camouflage index” if aggregation becomes needed. 

	•	RECONCILIATION

Definition
Reconciliation involves building or bridging divisive lines—seeking cooperation and shared goals with individuals from the “other side”—among individuals from groups that may be prone to separations by a tense or contentious boundary. It manifests in cross-group affiliations, co-authorships, or partnerships, reflecting an effort to foster harmony and reduce tension.

Theoretical Underpinnings
Reconciliation can emerge when individuals move beyond “us vs. them” categories and instead adopt a superordinate or bridging identity that highlights shared professional, organizational, or human commonalities (Gaertner and Dovidio, 2000). The emotional underpinnings of reconciliation stem from a desire to maintain harmonious relationships, restore trust, and reduce inter-group hostility (Ashforth, Harrison, and Corley, 2008). Efforts to build empathy or camaraderie align with more affect-based, relational elements of identity work. In particular, intergroup theories emphasize cooperative engagement as the clearest indication of conflict reduction (Allport, 1954; Pettigrew and Tropp, 2006). Forming partnerships across strongly demarcated lines constitutes a critical step in repairing social fractures (Williams, 2002; Hogg, 2016). In the aftermath of a sensitive or controversial event that heightens regional identities, reconciliation is therefore revealed in the decision to co-author, team up, or co-fund projects with out-group members.

Notable aspects of reconciliation include:

	•	Bridge-building. Reconciliation involves engaging with the “other side” (or the other group) to find areas of harmony, camaraderie, or collaboration.

	•	Addressing tension. While some individuals may consciously pursue reconciliation to heal rifts, others may find it easier or more rewarding to keep working relationships amicable and supportive, effectively bringing about reconciliation without an explicit agenda.

	•	Not necessarily a public gesture. Reconciliation can happen organically within small networks or lab groups rather than as a grand gesture intended for outsiders. The essence is a willingness to settle disputes or look past them, often through genuine personal connections, co-creation of knowledge, or shared problem-solving.

Current Empirical Operationalization
We capture reconciliation by measuring shifts in cross-regional-identity collaborations such as co-authorship or cooperative outputs. We compare the degree of mixed-group teamwork before and after a politically or socially salient event. If reconciliation occurs, we expect a notable uptick in cross-group teams in the post-event period. Under this conception, reconciliation is about pursuing initiatives with colleagues from the potential “other” group. 

Some Ideas for Website Cues
	•	Highlighted cross-group projects: Prominent listings of collaborations or joint grants with colleagues from different regions or groups, especially if such partnerships increased after a tense event.

	•	Positive, unifying language: Use of phrases like “joint efforts,” “bridging boundaries,” or “collaborating across diverse backgrounds” to signal inclusive outreach.

	•	Evidence of Outreach or Shared Initiatives: References to committees, workshops, or labs intentionally bringing together representatives from once-separate groups or institutions, pointing to purposeful relationship-building.

For example, we could measure (1) explicit bridging language emphasizing overlap or synergy among distinct groups (e.g., “Collaborating with Chinese and Hong Kong colleagues,” “Bringing together diverse perspectives”), or (2) inclusion of multiple affiliations that span different regions, signifying a shared identity beyond local boundaries.

We could also measure indirect clues, such as (3) mentions of cross-regional projects, committee memberships, or alliances, etc.

We could also measure researchers’ behavioral/social orientations (such as cooperative versus individualistic, etc.), although this may be seen as a more indirect measure/a substantial assumption about their behavioral tendencies. 

Your job is to figure out what these sort of these are there when looking at documents. Look closely at the words being used - small differences in language and workings can indicate a significant difference in mindset. Analyze the document critically.

Think step by step and write long, fully thought out analyses for each section to produce the most effective result possible. 

Here is the JSON response format that you should use to produce your analysis:

{
"general_analysis": "Your general analysis of the document and the text that was produced. Highlight key concepts relating to the website queues that are important",
"cross_group_thinking_ideas": "Your ideas when you are thinking about how the individual is talking about cross-group projects.",
"cross_group_thinking_score": "From a scale of 1-10, how much do you think the individual is thinking about cross-group projects. 10 being very high.",
"unifying_language_ideas": "Your ideas when you are thinking about how the individual is using unifying language.",
"unifying_language_score": "From a scale of 1-10, how much do you think the individual is using unifying language. 10 being very high.",
"shared_initiatives_outreach_ideas": "Your ideas when you are thinking about how the individual is talking about shared initiatives and outreach.",
"shared_initiatives_outreach_score": "From a scale of 1-10, how much do you think the individual is talking about reconciliation. 10 being very high.",
}
"""
        }, {
            "role": "user",
            "content": text
        }],
        response_format={
            "type": "json_object"
        },
        model="gpt-4o"
    )
    response = json.loads(response.choices[0].message.content)
    return response


In [ ]:
from rich import print
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# TODO: append university profile w/ personal website text
# TODO: make sure that results aren't correlated w/ more text (more personable because they wrote more) 

researcher_profile = []
def process_row(row):
    text = row['Personal Website']
    if len(text.strip()) == 0:
        return {
            "name": row["Name"],
            "error": "No text found"
        }
    response = process_homepage_text(text)
    response["name"] = row['Name']
    return response

max_threads = 10  # Set the maximum number of threads

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = [executor.submit(process_row, row) for index, row in df.iterrows()]
    for future in tqdm(futures):
        try:
            response = future.result()
            researcher_profile.append(response)
        except Exception as e:
            print(f"Error processing row: {e}")

  1%|          | 2/164 [00:11<15:00,  5.56s/it]

Error processing row: Expecting ':' delimiter: line 151 column 547 (char 58651)

  5%|▍         | 8/164 [06:12<2:12:57, 51.14s/it]

Error processing row: Expecting ':' delimiter: line 27184 column 1 (char 61430)

 10%|█         | 17/164 [06:58<53:02, 21.65s/it] 

...

Error processing row: Expecting ':' delimiter: line 1240 column 1 (char 43229)

 67%|██████▋   | 110/164 [08:31<02:11,  2.43s/it]

Error processing row: Error code: 400 - {'error': {'message': "We could not parse the JSON body of your request. 
(HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but 
what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our 
help center at help.openai.com.)", 'type': 'invalid_request_error', 'param': None, 'code': None}}

Error processing row: Error code: 400 - {'error': {'message': "We could not parse the JSON body of your request. 
(HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but 
what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our 
help center at help.openai.com.)", 'type': 'invalid_request_error', 'param': None, 'code': None}}

100%|██████████| 164/164 [08:31<00:00,  3.12s/it]


In [7]:
with open("homepage_data_analyzed.json", "w+") as f:
    json.dump(researcher_profile, f)

In [8]:
# TODO: save to 0721-month-analysis-data, save to new file